In [3]:
from grid import ipfsapi
import base64
import web3
import random

In [2]:
api = ipfsapi.connect('127.0.0.1', 5001)

In [28]:
def process_message(decoded):
    # send some sort of message
    try:
        # if message is a number... reply with the number + 1
        api.pubsub_pub(topic='openmined_model_1',payload=int(decoded['data']) + 1)
    except:
        print("could not accumulate... message not an int")
    print(decoded['data'])

# connect
g = api.pubsub_sub(topic='openmined_model_1',stream=True)

my_id = ""
secret = random.randint(0,1000000)

# send secret identity
api.pubsub_pub(topic='openmined_model_1',payload="id:" + str(secret))

results = list()
# try:
for encoded in g:
    
    if('from' in encoded):
        # decode message
        decoded = {}
        decoded['from'] = base64.standard_b64decode(encoded['from'])
        decoded['data'] = base64.standard_b64decode(encoded['data']).decode('ascii')
        decoded['seqno'] = base64.standard_b64decode(encoded['seqno'])
        decoded['topicIDs'] = encoded['topicIDs']
        decoded['encoded'] = encoded
        results.append(decoded)

        # If I don't know who I am - wait for id
        if(my_id == ""):
            if(str(decoded['data'].split(":")[-1]) == str(secret)):
                my_id = str(decoded['from'])
                print("Id:" + my_id)

        # if I know who i am - process message
        else:

            if(str(decoded['from']) != my_id and decoded['data'][0:3] != 'id:'):
                process_message(decoded)


Id:b'\x12 \xaety\x95\xb0\xeb\xd2\xb3f\xc9\xe1\xa9\x87\xf8(\xb0\x98cm\xa4\xd3=\xf3\x9a\xeax~\xce\x05mX['
could not accumulate... message not an int
your message good sir v2


KeyboardInterrupt: 